In [1]:
'''
 Davis, T., Love, B. C., & Preston, A. R. (2012). Striatal and hippocampal entropy 
 and recognition signals in category learning: Simultaneous processes revealed by 
 model-based fMRI. Journal of Experimental Psychology. Learning, Memory, and 
 Cognition, 38(4)
'''

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import sys
sys.path.append('..')
import sea
from tqdm import tqdm


In [2]:
# %% simulation parameters

param = {
    # how many steps one looksahead? 0 is myopic (1 step ahead) and so on..
    'MAX_RECURSION_LEVEL':  5,
    'EXPLORATION_PARAM':    0.0,
    'DECISION_PARAMETER':   1.0,
    'TOTAL_BLOCKS':         30,
    'CONSEC_BLOCKS':        2,
    'prior_matrix':         np.array([
                                [.01, .01],
                                [1.0, 1.0],
                                [1.0, 1.0],
                                [1.0, 1.0],
                                [1.0, 1.0]]),
    'num_dims':             5,
    'num_values':           2,
    'coupling':             .3,
    'd':                    1.0,
    'report':               False,
    'NUM_TIMES':            5,
}


LG_STIM = np.array([
    [0, 1, 1, 1, 1],  # exception A (first for later idx)
    [1, 0, 1, 1, 1],  # exception B (first for later idx)
    [0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0],
    [0, 0, 1, 0, 0],
    [1, 1, 0, 0, 1],
    [1, 1, 0, 1, 0],
    [1, 1, 1, 0, 0],])

LG_TRANS_STIM = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 1, 0],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 0, 0],
    [1, 1, 0, 1, 0],
    [1, 1, 0, 0, 1],])


In [3]:
# % functions for computing utility:

def getSituationCost(KNOWN_VEC, param):
    '''function to calculate costs for a given KNOWN_VEC (see RMC.SituationValue)

    This function assumes equal costs for each query. Can be modified for
    tasks in which some tests are more costly than others, or when sampling costs
    are not independent.'''

    return np.sum(KNOWN_VEC) * 10  # 10 is sampling cost used in paper


def dfActionVals(param):
    '''function to define utility table (e.g., table 1)

    When asym is True, this currently maximizes accuracy (i.e., correct=100
    utility units, incorrect=0 utility units).

    This function should be modified to reflect utility and cost associated
    with the final choice.'''

    m = np.diag([100] * param['num_values'])
    df = pd.DataFrame(index=['s%d' % a for a in range(param['num_values'])],
                      columns=['a%d' % a for a in range(param['num_values'])],
                      data=m)
    return df


param['getSituationCost'] = getSituationCost
param['dfActionVals'] = dfActionVals(param)


In [4]:
# function for creating results summary

def makeDf(learningSamples,learningAcc,itemRecog,transRecog):
    stimTypes = ['exception', 'rule', 'trans']
    dvs = ['learningSamples', 'learningAcc', 'itemRecog']
    dfRes = pd.DataFrame(index=stimTypes, columns=dvs)
    for stimType in stimTypes:
        if 'exception' in stimType:
            dfRes.loc[stimType, 'learningSamples'] = sum(
                learningSamples[0:2]) / (2. * param['TOTAL_BLOCKS'] * param['NUM_TIMES'])
            dfRes.loc[stimType, 'learningAcc'] = sum(
                learningAcc[0:2]) / (2. * param['TOTAL_BLOCKS'] * param['NUM_TIMES'])
            dfRes.loc[stimType, 'itemRecog'] = sum(
                itemRecog[0:2]) / (2. * param['NUM_TIMES'])
        elif 'rule' in stimType:
            dfRes.loc[stimType, 'learningSamples'] = sum(
                learningSamples[2:]) / (6. * param['TOTAL_BLOCKS'] * param['NUM_TIMES'])
            dfRes.loc[stimType, 'learningAcc'] = sum(
                learningAcc[2:]) / (6. * param['TOTAL_BLOCKS'] * param['NUM_TIMES'])
            dfRes.loc[stimType, 'itemRecog'] = sum(
                itemRecog[2:]) / (6. * param['NUM_TIMES'])
        elif 'trans' in stimType:
            dfRes.loc[stimType, 'itemRecog'] = sum(
                transRecog) / (8. * param['NUM_TIMES'])
    return dfRes

In [5]:
# %% Standard Model Simulation

model = sea.RMC(param)

LG_KNOWN = np.array([0, 1, 1, 1, 1])
LG_QUERY = np.array([1, 0, 0, 0, 0])
item_order = np.arange(len(LG_STIM))

learningSamples = np.zeros(len(LG_STIM))
learningAcc = np.zeros(len(LG_STIM))

itemRecog = np.zeros(len(LG_STIM))
transRecog = np.zeros(len(LG_TRANS_STIM))
clusterPerSim = []

allSamples = []


for run_num in tqdm(range(param['NUM_TIMES']),desc='run'):

    model.Reset()
    allSamples.append([])
    for num_block in range(
            param['TOTAL_BLOCKS']):
        np.random.shuffle(item_order)

        for item_num in item_order:
            tempN = model.PresentStimulus(
                LG_STIM[item_num], LG_KNOWN)  # ordered list to sample

            allSamples[run_num].append([item_num, tempN])

            SAMPLED_KNOWN = np.zeros(model.NUM_DIMS)
            SAMPLED_KNOWN[tempN] = 1

            correctProb = model.ResponseCorrectProb(
                LG_STIM[item_num], SAMPLED_KNOWN)

            learningSamples[item_num] += len(tempN)
            learningAcc[item_num] += correctProb

            model.Learn(LG_STIM[item_num], SAMPLED_KNOWN, LG_QUERY)

    clusterPerSim.append(len(model.clusterF) - 1)

    # Do Recognition
    for item in range(len(LG_STIM)):
        model.Stimulate(LG_STIM[item], LG_KNOWN)
        itemRecog[item] += sum(model.recog)

    for trans_item in range(len(LG_TRANS_STIM)):
        model.Stimulate(LG_TRANS_STIM[trans_item], LG_KNOWN)
        transRecog[trans_item] += sum(model.recog)


print("pickling allSamples to lgSample")
pickle.dump(allSamples, open("past_samples/rulex_%d_samples" % param['NUM_TIMES'], "wb"))


dfRes = makeDf(learningSamples,learningAcc,itemRecog,transRecog)

print('')
print(dfRes)


run: 100%|██████████| 5/5 [03:37<00:00, 44.23s/it]

pickling allSamples to lgSample

          learningSamples learningAcc  itemRecog
exception         3.13333    0.766983   0.106613
rule              2.44556    0.861106  0.0696617
trans                 NaN         NaN  0.0720091


In [6]:
# %% YOKED Model Simulation
model = sea.RMC(param)

LG_KNOWN = np.array([0, 1, 1, 1, 1])
LG_QUERY = np.array([1, 0, 0, 0, 0])
item_order = np.arange(len(LG_STIM))

learningSamples = np.zeros(len(LG_STIM))
learningAcc = np.zeros(len(LG_STIM))

itemRecog = np.zeros(len(LG_STIM))
transRecog = np.zeros(len(LG_TRANS_STIM))
clusterPerSim = []

pastSamples = pickle.load(open("past_samples/rulex_%d_samples" % param['NUM_TIMES'],'rb'))

for run_num in tqdm(range(param['NUM_TIMES']),desc='run'):

    model.Reset()
    trial = 0
    for num_block in range(param['TOTAL_BLOCKS']):

        np.random.shuffle(item_order)

        for item_num in item_order:
            tempN = pastSamples[run_num][trial][1]
            trial += 1
            SAMPLED_KNOWN = np.zeros(model.NUM_DIMS)
            SAMPLED_KNOWN[tempN] = 1
            model.Stimulate(LG_STIM[item_num], SAMPLED_KNOWN)

            correctProb = model.ResponseCorrectProb(
                LG_STIM[item_num], SAMPLED_KNOWN)

            learningSamples[item_num] += len(tempN)
            learningAcc[item_num] += correctProb

            model.Learn(LG_STIM[item_num], SAMPLED_KNOWN, LG_QUERY)

    # number of observations of each cluster/dimension/value combo + prior.
    # first dimension indexes cluster, 2nd the dimension, 3rd the dimension
    # value.
    clusterPerSim.append(len(model.clusterF) - 1)

    # Do Recognition
    for item in range(len(LG_STIM)):
        model.Stimulate(LG_STIM[item], LG_KNOWN)
        itemRecog[item] += sum(model.recog)

    for trans_item in range(len(LG_TRANS_STIM)):
        model.Stimulate(LG_TRANS_STIM[trans_item], LG_KNOWN)
        transRecog[trans_item] += sum(model.recog)


dfRes = makeDf(learningSamples,learningAcc,itemRecog,transRecog)

print('')
print(dfRes)



run: 100%|██████████| 5/5 [00:00<00:00, 17.17it/s]


          learningSamples learningAcc  itemRecog
exception         2.59667    0.569071   0.105942
rule              2.62444    0.808033  0.0757691
trans                 NaN         NaN  0.0680611
